
# IndabaX Zambia | Lusaka 2025

<img src="https://jacarandahealth.org/ypoagriw/2024/08/NIK3172-min-scaled.jpg" width="60%"/>


<!-- To include imahe -->

<!-- <a href="https://colab.research.google.com/drive/1jreqekFPmyaEH2ku58FrLI7MYfk34Uu4#scrollTo=lSZNitb4Yx1D" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a> -->

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1jreqekFPmyaEH2ku58FrLI7MYfk34Uu4#scrollTo=wWcHOdJ8YN0v)


© Dawa Health 2025.

Apache License 2.0.

**Authors:** Tafadzwa K. Munzwa, Tariro F. Munzwa, Kudzai Mwedzi

**Title:** Applied LLMs in Healthcare Building with RAG, Gemini, & UlizaLlama

---

**Introduction**

In this lesson, we demonstrate how Artificial Intelligence can be applied to real-world healthcare challenges faced in low-resource settings.
We’ll build a prototype conversational bot leveraging **UlizaLlama**, supporting English & Swahili (and potentially other languages). This bot can be used as a backend for WhatsApp messaging.

**Topics**

- Loading UlizaLlama  
- Prompt design / wrapper  
- Simple chat loop  
- Safety / filtering logic  
- (Discussion) WhatsApp integration

**Discussion:** Deploying AI solutions in low-resource environments



---


**Content:** <font color='green'>`AI for Health`</font> , <font color='orange'>`Conversational System`</font> ,

**Level:** <font color='grey'>`Beginner`</font>


**Aims / Learning Objectives**

By the end of this session, participants will be able to:

1. Load and use UlizaLlama in Colab  
2. Craft prompts / wrappers for health domain conversations  
3. Add basic safety filters  
4. Understand integration path to WhatsApp APIs  
---

**Prerequisites**

- Python familiarity  
- Basic knowledge of transformers / prompt engineering  
- Curiosity about digital health innovation

**Before you start:**

Run the "Installation and Imports" cell.


**🚀 Section 1 – Environment Setup**

In [ ]:
!pip install -q bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 15.3 MB/s eta 0:00:00


In [ ]:
import sys
required_version = (3, 11)
current_version = sys.version_info[:2]
if current_version != required_version:
    print(f"⚠️ Warning: Expected Python {required_version[0]}.{required_version[1]}, "
          f"but running {current_version[0]}.{current_version[1]}.")

!pip install uv -q
!uv pip install \
    transformers \
    accelerate \
    torch \
    langdetect \
    googletrans==4.0.0-rc1 \
    safetensors -q

from IPython.display import clear_output
clear_output()
print("✅ Dependencies installed")

✅ Dependencies installed


**💬 Section 2 – Load UlizaLlama model & tokenizer**

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch



model_name = "Jacaranda/UlizaLlama"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    load_in_8bit=True,  # Requires bitsandbytes
    device_map="auto"    # Requires accelerate
)

print("✅ UlizaLlama loaded in 8-bit")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/824 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/840k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggin

...

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.96G [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

✅ UlizaLlama loaded in 8-bit


🌍 **Section 3 - Basic Utility: detect language & translate**

In [ ]:

from langdetect import detect
from googletrans import Translator
translator = Translator()

def maybe_translate_to_english(text):
    src = detect(text)
    if src != "en":
        return translator.translate(text, src="auto", dest="en").text, src
    return text, "en"

def translate_back(text, target_lang):
    if target_lang != "en":
        return translator.translate(text, src="en", dest=target_lang).text
    return text



🤖 **Section 4 - Prompt / wrapper and safety**

In [ ]:
# You may define a system / role prompt
SYSTEM_PROMPT = (
    "You are a health assistant. You give general guidance, not medical diagnosis. "
    "Always include disclaimers. Keep the user safe."
)

def make_prompt(user_input):
    return (
        f"{SYSTEM_PROMPT}\n"
        f"User: {user_input}\n"
        f"Assistant:"
    )

def generate_response(prompt, max_new_tokens=100):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    out = model.generate(**inputs,
                         max_new_tokens=max_new_tokens,
                         temperature=0.7,
                         do_sample=True,
                         top_p=0.9)
    resp = tokenizer.decode(out[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)
    return resp


🔁 **Section 5 - Chat loop for testing**

In [ ]:
t

In [ ]:


def health_chat():
    print("💬 Uliza Health Bot prototype (type 'quit')\n")
    while True:
        user = input("You: ").strip()
        if user.lower() in ("quit", "exit"):
            break
        # Optionally translate
        text_en, src = maybe_translate_to_english(user)
        prompt = make_prompt(text_en)
        resp_en = generate_response(prompt)
        # Translate back if needed
        resp_out = translate_back(resp_en, src) if src != "en" else resp_en
        print(f"Bot ({src}): {resp_out}\n")

health_chat()


💬 Uliza Health Bot prototype (type 'quit')

Bot (cy): Rwy'n gwneud yn dda, beth amdanoch chi?

Bot (sw): Hello!Ninawezaje kukusaidia?

Bot (sw): Hakikisha kushauriana na daktari wako ili kuhakikisha kuwa unachukua hatua zinazofaa kukuweka salama na mtoto wako.

Bot (sw): Inategemea upendeleo wako wa lishe, malengo ya afya, na mzio wowote ambao unaweza kuwa nao.Ni bora kushauriana na mtaalamu wa huduma ya afya au mtaalam wa chakula aliyesajiliwa kupata ushauri wa kibinafsi.

Bot (sw): Mimba ya miezi mitano haiwezekani.Mimba ndefu zaidi ya mwanadamu inaweza kuwa miezi tisa.Ikiwa unajaribu kuwa mjamzito, ninapendekeza kushauriana na mtaalamu wa huduma ya afya.

Bot (en): I'm sorry, I'm not programmed to understand English. However, I can help you with translations between Swahili and English. Please provide me the sentences you would like to translate.

Bot (en): I'm sorry, but I'm unable to provide information on that topic.

Bot (en): If you experience strong or frequent contractions, c

🗣️ **Section 6 - (Discussion) WhatsApp Integration & Deployment**

- Use a Flask or FastAPI app that receives incoming WhatsApp messages (e.g. via Twilio or Meta Cloud API).

- For each message:

   1. Extract text
   2. Optionally translate / cleanup
   3. Call the LLM with your prompt wrapper
   4. Filter or sanitize the response
   5. Return the response as WhatsApp text

- Add rate limiting, logging, fallback if the model fails


---



🔐 **Section 7 - Safety, Testing & Fine-Tuning**

Before production, fine-tune UlizaLlama on medical / maternal-health conversational data using LoRA (as recommended by the model card).
Hugging Face

Add response filters (e.g. detect hallucinations, out-of-scope content)

Always include a disclaimer: “This is not a medical diagnosis; consult a licensed professional.”

Test extensively with domain experts